In [2]:
import torch
!ls ../../../.data/old

# Retriever convert (Martin Fajčik)

In [9]:
retriever_path_old = "../../../.data/old/dpr_official_questionencoder_nq.pt"
retriever_path_new = "../../../.data/dpr_official_questionencoder_nq.pt"

model_sd = torch.load(retriever_path_old, map_location=torch.device("cpu"))
model_cfg = {
    "model_type": "bert-base-uncased",
    "model_cache_dir": None,
    "use_projection": False,
}

checkpoint = {
    "model": model_sd,
    "config": model_cfg
}

torch.save(checkpoint, retriever_path_new)

In [14]:
from transformers import AutoConfig

reranker_path_old = "../../../.data/old/concat_reranker_longformer-base-4096_2020-12-18-12-02_pcknot4.ckpt_HIT@25_0.825168436679228.ckpt"
reranker_path_new = "../../../.data/concat_reranker_longformer-base-4096_2020-12-18-12-02_pcknot4.ckpt_HIT@25_0.825168436679228.ckpt"

config = AutoConfig.from_pretrained("allenai/longformer-base-4096", cache_dir="../../../.cache")

model_sd = torch.load(reranker_path_old, map_location=torch.device("cpu"))["state_dict"]
model_cfg = {
    "reranker_model_type": "concat",
    "encoder_config": config,
    "max_length": 4096,
    "passage_attention": False,
    "with_cls": False,
    "negatives": False,
    "training": {}
}

checkpoint = {
    "model": model_sd,
    "config": model_cfg
}

torch.save(checkpoint, reranker_path_new)

In [5]:
reranker_path_old = "../../../.data/old/reranker_roberta-base_2021-01-22-10-57_pcknot4_HIT@25_0.8355601233299075.ckpt"
reranker_path_new = "../../../.data/reranker_roberta-base_2021-01-22-10-57_pcknot4_HIT@25_0.8355601233299075.ckpt"

checkpoint = torch.load(reranker_path_old, map_location=torch.device("cpu"))
print(checkpoint["config"].keys())
checkpoint["config"]["encoder"] = "roberta-base"

torch.save(checkpoint, reranker_path_new)

dict_keys(['reranker_model_type', 'encoder_config', 'max_length', 'negatives', 'training'])


# Fusion Decoder

In [9]:
reranker_path_old = "/mnt/data/xondre09_pc5/.data/old/generative_reader_EM0.4499_S7000_Mt5-base_2020-12-23_10:25.pt"
reranker_path_new = "/mnt/data/xondre09_pc5/.data/generative_reader_EM0.4499_S7000_Mt5-base_2020-12-23_10:25.pt"

checkpoint = torch.load(reranker_path_old, map_location=torch.device("cpu"))

torch.save({
    "model": checkpoint["state_dict"],
    "config": {
        "encoder": "t5-base",
        "encoder_config": checkpoint["config"],
        "context_size": 25,
        "preprocessing_truncation": "truncate_only_passages",
        "fusion_strategy": "allinputs"
    }
}, reranker_path_new)